# Data Anonymization

In Pega CDH 8.5 and up, it's now possible to record the historical data as seen by the Adaptive Models. See [this academy challenge](https://academy.pega.com/challenge/exporting-historical-data/v4) for reference. This historical data can be further used to experiment with offline models, but also to fine-tune the OOTB Gradient Boosting model. However, sharing this information with Pega can be sensitive as it contains raw predictor data. 

To this end, we provide a simple and transparent script to fully anonimize this dataset.

The DataAnonymization script is now part of pdstools, and you can import it directly as such.

In [1]:
# These lines are only for rendering in the docs, and are hidden through Jupyter tags
# Do not run if you're running the notebook seperately

import os  
import sys
import plotly.io as pio
pio.renderers.default = "notebook_connected"

sys.path.append("../../../")
sys.path.append('../../python')

In [2]:
from pdstools import ADMDatamart
from pdstools import Config, DataAnonymization
import polars as pl

## Input data

To demonstrate this process, we're going to anonymise this toy example dataframe:

In [3]:
pl.read_ndjson('../../../../data/SampleHDS.json')

Context_Name,Customer_MaritalStatus,Customer_CLV,Customer_City,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,i64,str,str,str
"""FirstMortgage3…","""Married""",1460,"""Port Raoul""","""Account""","""Rejected"""
"""FirstMortgage3…","""Unknown""",669,"""Laurianneshire…","""AutoLoans""","""Accepted"""
"""MoneyMarketSav…","""No Resp+""",1174,"""Jacobshaven""","""Account""","""Rejected"""
"""BasicChecking""","""Unknown""",1476,"""Lindton""","""Account""","""Rejected"""
"""BasicChecking""","""Married""",1211,"""South Jimmiesh…","""DepositAccount…","""Accepted"""
"""UPlusFinPerson…","""No Resp+""",533,"""Bergeville""",null,"""Rejected"""
"""BasicChecking""","""No Resp+""",555,"""Willyville""","""DepositAccount…","""Rejected"""


As you can see, this dataset consists of regular predictors, IH predictors, context keys and the outcome column. Additionally, some columns are numeric, others are strings. Let's first initialize the DataAnonymization class.

In [4]:
anon = DataAnonymization(hds_folder='../../../../data/')

By default, the class applies a set of anonymisation techniques:
- Column names are remapped to a non-descriptive name
- Categorical values are hashed with a random seed
- Numerical values are normalized between 0 and 1
- Outcomes are mapped to a binary outcome.

To apply these techniques, simply call `.process()`:

In [5]:
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_PREDICTOR_0,Decision_Outcome
str,str,f64,str,str,str,bool
"""../../../../da…","""65732777866868…",0.983033,"""20880812621919…","""16833361426168…","""93311249197735…",false
"""../../../../da…","""82794960301423…",0.144221,"""11028392519531…","""16833361426168…","""11267924501229…",true
"""../../../../da…","""16845454466849…",0.679745,"""65941724532634…","""95646249209787…","""93311249197735…",false
"""../../../../da…","""41416529054668…",1.0,"""11028392519531…","""10452557567083…","""93311249197735…",false
"""../../../../da…","""17044567919853…",0.718982,"""20880812621919…","""10452557567083…","""63814384168383…",true
"""../../../../da…","""89888647483259…",0.0,"""65941724532634…","""10894427003473…",null,false
"""../../../../da…","""10579678940844…",0.02333,"""65941724532634…","""10452557567083…","""63814384168383…",false


To trace back the columns to their original names, the class also contains a mapping, which does not have to be provided.

In [6]:
anon.column_mapping

{'filename': 'filename',
 'Customer_City': 'PREDICTOR_1',
 'Customer_CLV': 'PREDICTOR_2',
 'Customer_MaritalStatus': 'PREDICTOR_3',
 'Context_Name': 'Context_Name',
 'IH_Web_Inbound_Accepted_pxLastGroupID': 'IH_PREDICTOR_0',
 'Decision_Outcome': 'Decision_Outcome'}

## Configs

Each capability can optionally be turned off - see below for the full list of config options, and refer to the API reference for the full description.

In [7]:
dict(zip(Config.__init__.__code__.co_varnames[1:], Config.__init__.__defaults__))

{'config_file': None,
 'hds_folder': '.',
 'use_datamart': False,
 'datamart_folder': 'datamart',
 'output_format': 'ndjson',
 'output_folder': 'output',
 'mapping_file': 'mapping.map',
 'mask_predictor_names': True,
 'mask_context_key_names': False,
 'mask_ih_names': True,
 'mask_outcome_name': False,
 'mask_predictor_values': True,
 'mask_context_key_values': True,
 'mask_ih_values': True,
 'mask_outcome_values': True,
 'context_key_label': 'Context_*',
 'ih_label': 'IH_*',
 'outcome_column': 'Decision_Outcome',
 'positive_outcomes': ['Accepted', 'Clicked'],
 'negative_outcomes': ['Rejected', 'Impression'],
 'special_predictors': ['Decision_DecisionTime',
  'Decision_OutcomeTime',
  'Decision_Rank'],
 'sample_percentage_schema_inferencing': 0.01}

It's easy to change these parameters by just passing the keyword arguments. In the following example, we
- Keep the IH predictor names
- Keep the outcome values
- Keep the context key values
- Keep the context key predictor names

In [8]:
anon = DataAnonymization(
    hds_folder="../../../../data/",
    mask_ih_names=False,
    mask_outcome_values=False,
    mask_context_key_values=False,
    mask_context_key_names=False,
)
anon.process()


filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../da…","""57519756201559…",0.983033,"""69711617015562…","""FirstMortgage3…","""13962235838568…","""Rejected"""
"""../../../../da…","""31859296589808…",0.144221,"""15548132004687…","""FirstMortgage3…","""29862476782353…","""Accepted"""
"""../../../../da…","""15095784979404…",0.679745,"""13434331885454…","""MoneyMarketSav…","""13962235838568…","""Rejected"""
"""../../../../da…","""61006127432065…",1.0,"""15548132004687…","""BasicChecking""","""13962235838568…","""Rejected"""
"""../../../../da…","""13360973173448…",0.718982,"""69711617015562…","""BasicChecking""","""12767155822890…","""Accepted"""
"""../../../../da…","""16008456843210…",0.0,"""13434331885454…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""14038953402839…",0.02333,"""13434331885454…","""BasicChecking""","""12767155822890…","""Rejected"""


The configs can also be written and read as such:

In [9]:
anon.config.save_to_config_file('config.json')

In [10]:
anon = DataAnonymization(config=Config(config_file='config.json'))
anon.process()

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../da…","""60226079550340…",0.983033,"""17806506984616…","""FirstMortgage3…","""15942892474020…","""Rejected"""
"""../../../../da…","""32983244890963…",0.144221,"""88158534262136…","""FirstMortgage3…","""17273842993958…","""Accepted"""
"""../../../../da…","""42674723931660…",0.679745,"""17327331774184…","""MoneyMarketSav…","""15942892474020…","""Rejected"""
"""../../../../da…","""92948969117739…",1.0,"""88158534262136…","""BasicChecking""","""15942892474020…","""Rejected"""
"""../../../../da…","""15892976791960…",0.718982,"""17806506984616…","""BasicChecking""","""37857970005736…","""Accepted"""
"""../../../../da…","""48562473446711…",0.0,"""17327331774184…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""31907197921853…",0.02333,"""17327331774184…","""BasicChecking""","""37857970005736…","""Rejected"""


## Exporting
Two functions export:
- `create_mapping_file()` writes the mapping file of the predictor names
- `write_to_output()` writes the processed dataframe to disk

Write to output accepts the following extensions: `["ndjson", "parquet", "arrow", "csv"]`

In [11]:
anon.create_mapping_file()
with open('mapping.map') as f:
    print(f.read())

filename=filename
Customer_City=PREDICTOR_1
Customer_CLV=PREDICTOR_2
Customer_MaritalStatus=PREDICTOR_3
Context_Name=Context_Name
IH_Web_Inbound_Accepted_pxLastGroupID=IH_Web_Inbound_Accepted_pxLastGroupID
Decision_Outcome=Decision_Outcome



In [12]:
anon.write_to_output(ext='arrow')

In [13]:
pl.read_ipc('output/hds.arrow')

PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,f64,str,str,str,str
"""94520044034088…",0.983033,"""14568678670821…","""FirstMortgage3…","""15332385152909…","""Rejected"""
"""17657541265028…",0.144221,"""15209912510629…","""FirstMortgage3…","""11038777584306…","""Accepted"""
"""19060241620686…",0.679745,"""81921098528204…","""MoneyMarketSav…","""15332385152909…","""Rejected"""
"""13649717206559…",1.0,"""15209912510629…","""BasicChecking""","""15332385152909…","""Rejected"""
"""56133732841987…",0.718982,"""14568678670821…","""BasicChecking""","""11628014325818…","""Accepted"""
"""58464924240807…",0.0,"""81921098528204…","""UPlusFinPerson…",null,"""Rejected"""
"""12697572995518…",0.02333,"""81921098528204…","""BasicChecking""","""11628014325818…","""Rejected"""


## Advanced: Hash fuctions

By default, we use [the same hashing algorithm Polars](https://pola-rs.github.io/polars/py-polars/html/reference/expressions/api/polars.Expr.hash.html#polars.Expr.hash) uses: [xxhash](https://github.com/Cyan4973/xxHash), as implemented [here](https://github.com/pola-rs/polars/blob/3f287f370b3c388ed2f3f218b2c096382548136f/polars/polars-core/src/vector_hasher.rs#L266). xxhash is fast to compute, and you can check its performance in collision, dispersion and randomness [here](https://github.com/Cyan4973/xxHash/tree/dev/tests). 

xxhash accepts four distinct seeds, but by default we set the seeds to `0`. It is possible to set the `seed` argument of the `process()` function to `'random'`, which will set all four seeds to a random integer between `0` and `1000000000`. Alternatively, it is possible to supply the four seeds manually with arguments `seed`, `seed_1`, `seed_2` and `seed_3`. 

If the xxhash with (random) seed(s) is not deemed sufficiently secure, it is possible to use your own hashing algorithm.

Note that since we're now running python code and not native Polars code anymore, this will be _significantly_ slower. Nonetheless, it is possible.

Just as an example - this is how one would use sha3_256:

In [14]:
from hashlib import sha3_256

anon.process(algorithm=lambda x: sha3_256(x.encode()).hexdigest())

filename,PREDICTOR_1,PREDICTOR_2,PREDICTOR_3,Context_Name,IH_Web_Inbound_Accepted_pxLastGroupID,Decision_Outcome
str,str,f64,str,str,str,str
"""../../../../da…","""c73023dda51de1…",0.983033,"""af486bc1815c72…","""FirstMortgage3…","""2513d0e24d8d02…","""Rejected"""
"""../../../../da…","""5414b197fb87b3…",0.144221,"""3cdf7b1ae1342e…","""FirstMortgage3…","""8b55bf62fc2ecd…","""Accepted"""
"""../../../../da…","""892f5d61a9215e…",0.679745,"""806ef34eb8237f…","""MoneyMarketSav…","""2513d0e24d8d02…","""Rejected"""
"""../../../../da…","""c3aa807d9e927a…",1.0,"""3cdf7b1ae1342e…","""BasicChecking""","""2513d0e24d8d02…","""Rejected"""
"""../../../../da…","""c2d492f9a05073…",0.718982,"""af486bc1815c72…","""BasicChecking""","""1cd082ecbd5dd9…","""Accepted"""
"""../../../../da…","""b21059221605c4…",0.0,"""806ef34eb8237f…","""UPlusFinPerson…",null,"""Rejected"""
"""../../../../da…","""80cbe489dcecd4…",0.02333,"""806ef34eb8237f…","""BasicChecking""","""1cd082ecbd5dd9…","""Rejected"""
